In [138]:
import json
from rdflib import Graph
import pandas as pd
import random

In [153]:
g = Graph()
g.parse("sto/sto-enriched.nt", format="nt")
    
len(g) # prints 2
    
qres = g.query(
    """PREFIX owl: <http://www.w3.org/2002/07/owl#>
       PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
       PREFIX sto: <https://w3id.org/i40/sto#>
    
    select ?s where {
            ?s rdf:type sto:Standard .
    } limit 1000""")

        
entities = pd.DataFrame(columns=['s'])
obj = []
for row in qres:
    obj.append(str("%s" %row))
entities.s=obj


In [154]:
count = 0
list_pd = pd.DataFrame(columns=['s', 'p', 'o'])
for i in range(entities.shape[0]):
    standard = entities.s[i]
    #print(type(entities.s[i]))
    qres = g.query(
        """PREFIX owl: <http://www.w3.org/2002/07/owl#>
            PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
            PREFIX sto: <https://w3id.org/i40/sto#>
            select distinct ?o where {
                <""" +standard+ """> sto:relatedTo ?o .
            }""")
    lst = pd.DataFrame(columns=['s', 'p', 'o'])
    obj = []
    for row in qres:
        obj.append(str("%s" %row))
    
    if len(obj)>0:
        #print((type(standard)))
        lst.o=obj
        lst.s=standard
        lst.p='https://w3id.org/i40/sto#relatedTo'
        list_pd = pd.concat([list_pd, lst], ignore_index=True)
    else:
        count+=1
#list_pd = list_pd.drop_duplicates().reset_index(drop=True)
    

In [155]:
n_select= round(list_pd.shape[0]/10)
print(n_select)
index = list_pd.index
sampling = random.choices(index, k=n_select)

65


In [156]:
test_set=list_pd.iloc[sampling]
duplicateRowsDF = test_set[test_set.duplicated()]
test_set = test_set.drop_duplicates().reset_index(drop=True)
print(test_set.shape[0])
duplicateRowsDF

60


,s,p,o
405,https://w3id.org/i40/sto#MathML,https://w3id.org/i40/sto#relatedTo,https://w3id.org/i40/sto#ISO_20922
127,https://w3id.org/i40/sto#ISO_15926,https://w3id.org/i40/sto#relatedTo,https://w3id.org/i40/sto#IEC_61499
301,https://w3id.org/i40/sto#DIN_SPEC_16592,https://w3id.org/i40/sto#relatedTo,https://w3id.org/i40/sto#ISO_29002
426,https://w3id.org/i40/sto#eClass,https://w3id.org/i40/sto#relatedTo,https://w3id.org/i40/sto#ISO_15746
505,https://w3id.org/i40/sto#IEC_61804,https://w3id.org/i40/sto#relatedTo,https://w3id.org/i40/sto#IEC_61804


In [157]:
dataset = pd.DataFrame(columns=['s', 'p', 'o'])
qres = g.query(
    """PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX sto: <https://w3id.org/i40/sto#>
        select distinct ?s ?p ?o where {
            ?s ?p ?o .
        }""")
sub = []
prop = []
obj = []
for row in qres:
    sub.append(str("%s" %row['s']))
    prop.append(str("%s" %row['p']))
    obj.append(str("%s" %row['o']))
dataset.s=sub
dataset.p=prop
dataset.o=obj

In [158]:
duplicateRowsDF = dataset[dataset.duplicated()]
dataset = dataset.drop_duplicates().reset_index(drop=True)
print(dataset.shape[0])
duplicateRowsDF

21148


,s,p,o
18036,https://w3id.org/i40/sto#OpenFogConsortium,http://www.w3.org/2000/01/rdf-schema#label,OpenFog Consortium


In [162]:
test_set = dataset[~dataset.apply(tuple,1).isin(test_set.apply(tuple,1))]

In [164]:
dataset.shape[0]-test_set.shape[0]

60